In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenmake as s

In [ ]:
import pint

u = pint._DEFAULT_REGISTRY

In [ ]:
def plot_cell(cell, exclude=(2,)):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    poly_dict = cell.get_polygons(by_spec=True)
    plt.figure(figsize=(20, 12))
    for layer_datatype, polys in poly_dict.items():
        if layer_datatype[0] in exclude:
            continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="k", fc=(1, 0, 0, 0.5))
    plt.axes().set_aspect("equal", "datalim")


def write_gds(main_cell, filename, unit=1.0e-6, precision=1.0e-9):
    writer = gdspy.GdsWriter(filename, unit=unit, precision=precision)
    for cell in [main_cell] + list(main_cell.get_dependencies(True)):
        writer.write_cell(cell)
    writer.close()

# Single-input prototype

In [ ]:
metadata = {}
chip_dims = np.array([40e3, 20e3])
main_cell = s.chip(
    "Big Snake",
    s.manifold_snake,
    dims=chip_dims,
    split=None,
    num_inputs=1,
    lanes_per_snake=5,
    manifold_margin=0.5e3,
    manifold_radius=2e3,
    feeding_channel_width=400,
    trench_spacing=2,
    trench_gap=20,
    trench_width=1.5,
    port_margin=1.5e3,
    top_margin=1.2e3,
    bottom_margin=1.2e3,
    draw_trenches=False,
    metadata=metadata,
)
print(metadata)
plot_cell(main_cell)

# Flow calculations

In [ ]:
eta = 1 * u.centipoise

In [ ]:
def Q_over_deltap(eta, L, h, w):
    h_ = np.minimum(h, w)
    w_ = np.maximum(h, w)
    h, w = h_, w_
    return h**3 * w / (12 * eta * L) * (1 - 0.630 * h / w)

In [ ]:
baseline_md = chip_metadata(feeding_channel_width=150, lane_gap=400)
print(baseline_md)

In [ ]:
h = 50 * u.um
w = baseline_md["feeding_channel_width"] * u.um
L = baseline_md["split"][0] * baseline_md["lane_length"] * u.um
baseline_Q_over_deltap = Q_over_deltap(eta, L, h, w)

In [ ]:
h = 50 * u.um
w = np.linspace(50, 150, 100) * u.um
L = 15 * 17000 * u.um
plt.plot(w, baseline_Q_over_deltap / Q_over_deltap(eta, L, h, w))

In [ ]:
np.array((5056, 2960)) * 4.25 / 20

In [ ]:
lane_length = 17615 * u.um
h = 50 * u.um
w = np.linspace(50, 150, 100) * u.um
plt.figure(figsize=(10, 8))
ax = plt.gca()
ax.plot(w, 1 / Q_over_deltap(eta, lane_length * 23, 50, w))
for (lanes, width, height), c in zip(
    ((15, 50, 50), (19, 90, 90), (29, 45, 90), (23, 40, 50), (23, 40, 90)),
    plt.cm.get_cmap("Set1").colors,
):
    v = 1 / Q_over_deltap(eta, lane_length * lanes, height, width)
    ax.axhline(
        v.magnitude,
        c=c,
        ls="--",
        label="{} lane snake ({} um width, {} um height)".format(lanes, width, height),
    )
plt.legend()

In [ ]:
plt.figure(figsize=(10, 8))
hs = np.array([50, 75, 100, 150]) * u.um
w = np.linspace(40, 150, 100) * u.um
chips = [
    chip_metadata(feeding_channel_width=width.magnitude, trench_spacing=2, split=8)
    for width in w
]
L = np.array([np.max(m["split"]) * m["lane_length"] for m in chips]) * u.um
# L = 15 * 17000 * u.um
eta = 1 * u.centipoise
ax = plt.gca()
for h in hs:
    ax.plot(
        w,
        baseline_Q_over_deltap / Q_over_deltap(eta, L, h, w),
        label="feeding channel height: {:~P}".format(h),
    )
ax2 = plt.gca().twinx()
ax2.plot(
    w,
    [m["num_trenches"] for m in chips],
    ls="-.",
    c="purple",
    label="number of trenches (2 um trench spacing)",
)
# ax2.plot(w, [np.max(m['split']) for m in chips], c='r', label='number of lanes')
ax.axhline(1, c="k", ls=":", label="15 lane snake (50 um height)")
ax.legend()
ax.set_ylabel("pressure (relative to 15-lane snake)")
ax.set_xlabel("feeding channel width (um)")
ax2.legend(loc="lower left")

# Designs

In [ ]:
def chip_metadata(**kwargs):
    metadata = {}
    main_cell = s.chip(
        "my_chip", **{"metadata": metadata, **kwargs, "draw_trenches": False}
    )
    return metadata["my_chip"]

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims0 = np.array([23e3, 13e3])
chip_dims1 = np.array([40e3, 20e3])
chip_dims2 = np.array([25e3, 16e3])

In [ ]:
plt.figure(figsize=(10, 5))
ax = plt.gca()
ax.add_patch(plt.Rectangle(-coverslip_dims / 2, *coverslip_dims, fill=False))
ax.add_patch(plt.Rectangle(-chip_dims0 / 2, *chip_dims0, fill=False, ls="-."))
ax.add_patch(plt.Rectangle(-chip_dims1 / 2, *chip_dims1, fill=False, ls="--"))
ax.add_patch(plt.Rectangle(-chip_dims2 / 2, *chip_dims2, fill=False))
ax.set_xlim(-1.2 * coverslip_dims[0] / 2, 1.2 * coverslip_dims[0] / 2)
ax.set_ylim(-1.2 * coverslip_dims[1] / 2, 1.2 * coverslip_dims[1] / 2)
ax.set_aspect("equal")

In [ ]:
def fovs_per_chip(
    fov,
    feeding_channel_width=None,
    trench_length=None,
    trench_gap=None,
    num_lanes=None,
    lane_with_trenches_length=None,
    **kwargs,
):
    hfovs = int(np.ceil(lane_with_trenches_length / fov[0]))
    y = 2 * trench_length + trench_gap
    unit_cell_height = y + feeding_channel_width
    trench_sets_per_fov = 2
    while True:
        delta_y = feeding_channel_width + trench_length
        if y + delta_y < fov[1]:
            y += delta_y
            unit_cell_height = y + trench_gap
            trench_sets_per_fov += 1
        else:
            break
        delta_y = trench_gap + trench_length
        if y + delta_y < fov[1]:
            y += delta_y
            unit_cell_height = y + feeding_channel_width
            trench_sets_per_fov += 1
        else:
            break
    vfovs = int(np.ceil(num_lanes * 2 / trench_sets_per_fov))
    return np.array([hfovs, vfovs]), unit_cell_height, y

In [ ]:
fov = np.array([5056, 2960]) * 4.25 / 20

In [ ]:
metadata = {}
main_cell = s.chip(
    "Big Snake",
    s.snake,
    dims=chip_dims2,
    split=8,
    gap_lanes=1,
    feeding_channel_width=80,
    trench_spacing=2,
    trench_gap=20,
    trench_width=1.5,
    port_margin=1.5e3,
    top_margin=1.2e3,
    bottom_margin=1.2e3,
    horizontal_margin=4e3,
    draw_trenches=False,
    metadata=metadata,
)
fovs, unit_cell_height, active_height = fovs_per_chip(fov, **metadata["Big Snake"])
print(metadata)
print("FOVs: {} ({} x {})".format(np.product(fovs), fovs[0], fovs[1]))
print("unit cell height:", unit_cell_height)
print("active height: {} (margin: {})".format(active_height, fov[1] - active_height))
plot_cell(main_cell)

In [ ]:
# height: 1.4
# x2 S8 W1.3 L35 TS1
# x2 S8 W1.5 L35 TS1
# x1 S8 W1.5 L35 TS3
# x1 S8 W1.5 L45 TS2

# S8 W1.5 L35-65 sampler?

In [ ]:
%%time
metadata = {}
base_params = dict(
    dims=chip_dims2,
    split=24,
    gap_lanes=0,
    trench_length=35,
    feeding_channel_width=40,
    trench_gap=20,
    port_margin=1.5e3,
    top_margin=1.2e3,
    bottom_margin=1.2e3,
    horizontal_margin=4e3,
    draw_trenches=True,
    metadata=metadata,
)
params = [
    dict(trench_width=1.4, trench_spacing=2.1, **base_params),
    dict(trench_width=1.4, trench_spacing=2.1, **base_params),
    dict(trench_width=1.4, trench_spacing=1.1, **base_params),
    {
        **base_params,
        **dict(
            trench_width=1.5,
            trench_spacing=2.1,
            trench_length=45,
            trench_gap=15,
            feeding_channel_width=70,
            top_margin=1e3,
            bottom_margin=1e3,
        ),
    },
    dict(trench_width=1.2, trench_spacing=2.1, **base_params),
    dict(trench_width=1.2, trench_spacing=2.1, **base_params),
]
# params = [dict(split=4, fc=40, w=1.4, ts=1.6, tg=10), dict(split=4, fc=40, w=1.4, ts=1.1, tg=10), dict(split=4, fc=40, w=1.4, ts=1.1, tg=10),
#          dict(split=8, fc=40, w=1.4, ts=1.6, tg=10), dict(split=8, fc=40, w=1.4, ts=1.1, tg=10), dict(split=8, fc=40, w=1.4, ts=1.1, tg=10)]
chips = [
    s.chip(
        "Basilisk S{split} FC{feeding_channel_width} L{trench_length} W{trench_width} TS{trench_spacing}".format(
            **params
        ),
        s.snake,
        **params,
    )
    for params in params
]
# chips = [chips[0]]*6
main_cell = s.wafer(
    chips, "Basilisk \n YG/JQS 191212", text=True, mask=False, chip_area_margin=1e3
)
write_gds(main_cell, "200128basilisk.gds")

In [ ]:
# print text summary for wafer spreadsheet

In [ ]:
plot_cell(main_cell)

In [ ]:
metadata

In [ ]:
plot_cell(chips[5])